# 读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## (**加载和保存张量**)

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。


In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

我们现在可以将存储在文件中的数据读回内存。


In [2]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

我们可以[**存储一个张量列表，然后把它们读回内存。**]


In [3]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以(**写入或读取从字符串映射到张量的字典**)。
当我们要读取或写入模型中的所有权重时，这很方便。


In [4]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## [**加载和保存模型参数**]

保存单个权重向量（或其他张量）确实有用，
但是如果我们想保存整个模型，并在以后加载它们，
单独保存每个向量则会变得很麻烦。
毕竟，我们可能有数百个参数散布在各处。
因此，深度学习框架提供了内置函数来保存和加载整个网络。
需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。
例如，如果我们有一个3层多层感知机，我们需要单独指定架构。
因为模型本身可以包含任意代码，所以模型本身难以序列化。
因此，为了恢复模型，我们需要用代码生成架构，
然后从磁盘加载参数。
让我们从熟悉的多层感知机开始尝试一下。


In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]


In [6]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)


In [7]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的`X`时，
两个实例的计算结果应该相同。
让我们来验证一下。


In [8]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。



## 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？
1. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？
1. 如何同时保存网络架构和参数？需要对架构加上什么限制？



## 练习解答


### 1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？


1. **备份和恢复**：存储模型参数可以作为备份，以防在后续的训练或开发过程中出现意外情况导致模型丢失或损坏。
2. **模型共享**：保存的模型参数可以与他人共享，使得其他研究人员或开发者能够在相同或不同的环境中复用和验证模型的性能。
3. **持续训练**：在长期的项目中，可能需要分阶段进行模型训练。保存模型参数允许开发者在未来的时间点继续之前的训练过程，而不是从头开始。
4. **模型比较**：保存不同时间点或不同超参数下的模型参数，可以帮助研究者比较和分析哪些条件下的模型表现最佳。
5. **迁移学习**：保存的模型参数可以用作预训练模型的基础，在此基础上进行进一步的微调或扩展，以适应新的任务或数据集。
6. **实验复现**：在进行科学研究时，保存模型参数是实验复现的重要部分，它确保了研究的可重复性。
7. **性能基准**：保存的模型参数可以用来建立性能基准，用于评估未来模型改进的效果。
8. **减少训练时间**：对于大型和复杂的模型，重新训练可能需要很长时间。保存模型参数可以避免不必要的重复训练，节省时间和计算资源。

### 2. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？

如果你想在一个新的网络中使用之前网络的前两层，可以通过以下步骤来实现：
1. 加载预训练模型：
   首先，你需要加载包含你想要复用的前两层结构的预训练模型。
2. 提取前两层：
   从预训练模型中提取出前两层，这通常涉及到选择模型的特定子模块。
3. 创建新网络：
   设计一个新的网络架构，这个架构应该在你想要插入的位置包含一个占位符，用于稍后插入提取的前两层。
4. 插入提取的层：
   将提取的前两层插入到新网络的相应位置。这可能需要一些额外的代码来确保层的连接是正确的。
5. 调整新网络：
   如果需要，对新网络的其他部分进行调整，以确保整个网络的兼容性和功能性。
6. 冻结前两层参数：
   如果你不想在训练新网络时改变前两层的学习参数，可以选择冻结这些层的权重。
7. 训练新网络：
   使用新数据集对新网络进行训练，此时可以选择只训练新添加的部分，或者同时训练所有可学习的层。

以下是一个简化的代码示例，展示了如何在PyTorch中实现这个过程：
```python
import torch
import torch.nn as nn
# 假设有一个预训练模型
pretrainedmodel = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# 提取前两层
layerstotransfer = list(pretrainedmodel.children())[:2]
# 创建一个新的网络架构
class NewNetwork(nn.Module):
    def init(self, layerstotransfer):
        super(NewNetwork, self).init()
      #   将提取的层插入到新网络中
        self.featureextractor = nn.Sequential(layerstotransfer)
      #   定义新网络的其他层
        self.newlayers = nn.Sequential(
            # 添加新的层
        )
    def forward(self, x):
        x = self.featureextractor(x)
        x = self.newlayers(x)
        return x
# 实例化新网络，并插入提取的层
newnetwork = NewNetwork(layerstotransfer)
# 冻结提取的层的参数
for param in newnetwork.featureextractor.parameters():
    param.requiresgrad = False
# 训练新网络...
```
在这个例子中，我们首先加载了一个预训练的ResNet-18模型，然后提取了它的前两层。接着，我们定义了一个新的网络类NewNetwork，在其中插入了提取的层，并添加了新的层。最后，我们冻结了提取的层的参数，这样在训练新网络时，这些层的权重不会更新。

### 3. 如何同时保存网络架构和参数？需要对架构加上什么限制？


在PyTorch中，你可以通过两种方式保存网络架构和参数：

保存整个模型：
你可以直接保存整个模型对象，这将包括模型的结构和参数。这样做的一个缺点是它会保存模型的完整状态，包括任何可能的额外属性或缓冲区，这可能会导致较大的文件大小。
python
torch.save(model, 'model_architecture_and_parameters.pth')
分别保存结构和参数：
另一种方法是单独保存模型的结构和参数。这种方法可以让你更灵活地控制加载过程，例如，当你想要用一个预训练的模型作为基础，并在此基础上添加新的层时。
```python
# 保存模型结构为Python脚本
with open('model_architecture.py', 'w') as f:
    f.write(model.__class__.__name__ + '(\n')
    for name, module in model._modules.items():
        f.write(f'    {name}={module.__class__.__name__}(\n')
        for param_name, param in module._parameters.items():
            f.write(f'        {param_name}={param.data.numpy().tolist()},\n')
        f.write('    ),\n')
    f.write(')\n')

# 保存模型参数
torch.save(model.state_dict(), 'model_parameters.pth')
对于第二种方法，你需要确保模型的结构是可序列化的，这意味着模型的定义可以被转换为文本形式并保存到一个文件中。在上面的示例中，我们通过遍历模型的所有模块和参数，将它们的名称和值写入到一个Python脚本文件中。这样，你就可以在另一个脚本中导入这个文件，并使用其中的定义来重建模型的结构。

当加载模型时，你需要先加载模型的结构，然后加载参数：

python
# 导入模型结构
from model_architecture import MyModelClass

# 创建模型实例
model = MyModelClass()

# 加载模型参数
model.load_state_dict(torch.load('model_parameters.pth'))
请注意，这种方法需要你手动管理模型结构的保存和加载，但它提供了更大的灵活性，特别是在处理复杂模型或需要自定义加载逻辑的情况下。

[Discussions](https://discuss.d2l.ai/t/1839)
